Connected to dl_312_venv (Python 3.12.0)

In [1]:
import sys

sys.path.append('..')

import pathlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ngram_model.utils import build_corpus, get_id_to_word, get_word_to_id_from_vocab, build_vocabulary
from word2vec.simple_cbow import SimpleCBOW
from common.trainers import Trainer
from common.optimizers import Adam
from common.utils import create_contexts_and_targets, convert_one_hot
from ngram_model.utils import tokenize, build_vocabulary, get_word_to_id_from_vocab, get_id_to_word, build_corpus
from word2vec.simple_cbow_pytorch import compute_loss, training_loop,SimpleCBOWPT, CBOWDataset,plot_losses,loss_fn

window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 100

text = "Prince loved princess from the bottom of his heart"

words = tokenize(text)
vocab = build_vocabulary(words)
word_to_id = get_word_to_id_from_vocab(vocab)
id_to_word = get_id_to_word(word_to_id)
corpus = build_corpus(words, word_to_id)
vocab_size = len(word_to_id)

contexts, targets = create_contexts_and_targets(corpus, window_size)
contexts = convert_one_hot(np.array(contexts), vocab_size)
targets = convert_one_hot(np.array(targets), vocab_size)

contexts = torch.tensor(contexts, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

cbow_dataset = CBOWDataset(contexts, targets)
cbow_dataloader = DataLoader(cbow_dataset, batch_size, shuffle=True)

cbow_model = SimpleCBOWPT(vocab_size, hidden_size)
optimizer = optim.Adam(cbow_model.parameters(),lr=0.001)
losses = training_loop(max_epoch, optimizer, cbow_model, loss_fn, cbow_dataloader, cbow_dataloader, pathlib.Path.cwd()/"checkpoint")
plot_losses(losses)

Epoch loop:   0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [2]:
for idx, (batch_x, batch_y) in enumerate(cbow_dataloader):
    scores = cbow_model(batch_x)
    print(f"scores shape : {scores.size()}")
    print(f"batch_y shape : {batch_y.size()}")
    loss = loss_fn(scores, batch_y)
    if idx>3:
        break

scores shape : torch.Size([3, 9])
batch_y shape : torch.Size([3, 9])
scores shape : torch.Size([3, 9])
batch_y shape : torch.Size([3, 9])
scores shape : torch.Size([1, 9])
batch_y shape : torch.Size([1, 9])
